Importation des library

In [ ]:
import pandas as pd
import numpy as np
import vectorbt as vbt
import matplotlib.pyplot as plt
import anywidget
import os
plt.style.use('dark_background')

Chargement des données

In [ ]:
# Télécharger les données (Utilise Python Binance)
# a PARTIR D'une date jusqu'a aujourd'hui (intervale 15 minutes)
symbol = ['ETHUSDT']
#day
time = "5m"
# Téléchargement des données
data = vbt.BinanceData.download(
    symbol, 
    start='2025-11-01', 
    interval=time,
    tz_convert='Europe/Paris',
)

Mise en forme des données

In [ ]:
# Agencement des données + index + mis à l'heure de paris
df_raw = data.get(['Close','Open','High','Low','Volume']).copy()
df_raw.tail()

In [ ]:
# Separation des données en train et test (IS/OOS) (70/30%)
split_idx = int(0.7*len(df_raw))
train_data = df_raw.iloc[:split_idx]
test_data = df_raw.iloc[split_idx:]

# Affichage de la séparation des données
plt.figure(figsize=(10, 1))
plt.barh(['Dataset'], [len(df_raw)], color='lightgray', alpha=0.3)
plt.barh(['Dataset'], [split_idx], color='blue', alpha=0.5, label='Train')
plt.barh(['Dataset'], [len(df_raw)-split_idx], left=split_idx, color='orange', alpha=0.5, label='Test')
plt.title('Train/Test Split Visualization')
plt.legend()
plt.show()

In [ ]:
# Selection du train_set comme données d'entrainement
df = train_data.copy()

In [ ]:
import vectorbt as vbt
import pandas as pd
import plotly.graph_objects as go

close = df['Close']

# ================================================
# 1) Indicateurs SMA
# ================================================
sma9 = vbt.MA.run(close, window=9).ma
sma21 = vbt.MA.run(close, window=21).ma
sma50 = vbt.MA.run(close, window=50).ma


# ================================================
# 2) Signaux basés UNIQUEMENT sur les croisements
# ================================================
entries = sma9.vbt.crossed_above(sma50)
exits   = sma9.vbt.crossed_below(sma50)

# ================================================
# 3) Backtest
# ================================================
pf = vbt.Portfolio.from_signals(
    close,
    entries,
    exits,
    init_cash=500,
    fees=0.0004,
    slippage=0.0002,
    freq='5m'
)

# ================================================
# 5) Résultats
# ================================================
print(pf.stats())

# ================================================
# 6) Chart : Candles + SMA + Signaux
# ================================================
fig = go.Figure()

# --- Bougies
fig.add_trace(go.Candlestick(
    x=df.index,
    open=df['Open'],
    high=df['High'],
    low=df['Low'],
    close=close,
    name="Bougies",
    increasing_line_color='rgba(0,200,0,0.6)',  # vert transparent
    decreasing_line_color='rgba(200,0,0,0.6)',  # rouge transparent
    increasing_fillcolor='rgba(0,200,0,0.3)',
    decreasing_fillcolor='rgba(200,0,0,0.3)'
))

# --- Moyennes mobiles
fig.add_trace(go.Scatter(x=df.index, y=sma9, mode="lines", name="SMA 9", line=dict(color='green')))
fig.add_trace(go.Scatter(x=df.index, y=sma21, mode="lines", name="SMA 21", line=dict(color='blue')))
fig.add_trace(go.Scatter(x=df.index, y=sma50, mode="lines", name="SMA 50", line=dict(color='orange')))


# --- Signaux Entrées
fig.add_trace(go.Scatter(
    x=df.index[entries],
    y=close[entries],
    mode='markers',
    marker=dict(color='lime', size=8),
    name='Buy'
))

# --- Signaux Sorties
fig.add_trace(go.Scatter(
    x=df.index[exits],
    y=close[exits],
    mode='markers',
    marker=dict(color='red', size=8),
    name='Sell'
))

# --- Paramètres graphiques
fig.update_layout(
    title="Stratégie SMA Cross — Bougies + Signaux",
    xaxis_title="Date",
    yaxis_title="Prix",
    xaxis_rangeslider_visible=False,
    template="plotly_dark",
    height=800
)

fig.show()
pf.plot().show()
